<a href="https://colab.research.google.com/github/dh266/Beach_House/blob/main/INT3405E_Group_Project_Modelratinguser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#genre classification with rating, user

## Part 0. Book

In [ ]:
import pandas
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import cv2
import os
from nltk import wordpunct_tokenize
import re

In [ ]:
!gdown 1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD

Downloading...
From: https://drive.google.com/uc?id=1hUqu1mbFeTEfBvl-7fc56fHFfCSzIktD
To: /content/ml1m.zip
100% 105M/105M [00:01<00:00, 96.2MB/s]


In [ ]:
!unzip -qq ml1m.zip -d ml1m

In [ ]:
# You need extract file ml1m.zip to folder ml1m before run code

users = pandas.read_csv('ml1m/content/dataset/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pandas.read_csv('ml1m/content/dataset/ratings.dat', engine='python',
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies_train = pandas.read_csv('ml1m/content/dataset/movies_train.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_test = pandas.read_csv('ml1m/content/dataset/movies_test.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')
movies_train['genre'] = movies_train.genre.str.split('|')
movies_test['genre'] = movies_test.genre.str.split('|')

users.age = users.age.astype('category')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('category')
ratings.userid = ratings.userid.astype('category')


In [ ]:
movies_train

,title,genre
movieid,,
1650,Washington Square (1997),[Drama]
185,"Net, The (1995)","[Sci-Fi, Thriller]"
1377,Batman Returns (1992),"[Action, Adventure, Comedy, Crime]"
3204,"Boys from Brazil, The (1978)",[Thriller]
1901,Dear Jesse (1997),[Documentary]
...,...,...
2539,Analyze This (1999),[Comedy]
3038,"Face in the Crowd, A (1957)",[Drama]
1832,Heaven's Burning (1997),"[Action, Drama]"


In [ ]:
# Ghép dữ liệu theo trường userid
ratingusers_data = pandas.merge(ratings, users, on='userid')
gender_mapping = {'F': 0, 'M': 1}
ratingusers_data['gender'] = ratingusers_data['gender'].map(gender_mapping)
# Hiển thị dữ liệu ghép
ratingusers_data

,userid,movieid,rating,timestamp,gender,age,occupation,zip
0,1,1193,5,978300760,0,1,10,48067
1,1,661,3,978302109,0,1,10,48067
2,1,914,3,978301968,0,1,10,48067
3,1,3408,4,978300275,0,1,10,48067
4,1,2355,5,978824291,0,1,10,48067
...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,1,25,6,11106
1000205,6040,1094,5,956704887,1,25,6,11106
1000206,6040,562,5,956704746,1,25,6,11106
1000207,6040,1096,4,956715648,1,25,6,11106


In [ ]:
ratingusers_data_train = pandas.merge(ratingusers_data, movies_train, on='movieid')
ratingusers_data_train

,userid,movieid,rating,timestamp,gender,age,occupation,zip,title,genre
0,1,1193,5,978300760,0,1,10,48067,One Flew Over the Cuckoo's Nest (1975),[Drama]
1,2,1193,5,978298413,1,56,16,70072,One Flew Over the Cuckoo's Nest (1975),[Drama]
2,12,1193,4,978220179,1,25,12,32793,One Flew Over the Cuckoo's Nest (1975),[Drama]
3,15,1193,4,978199279,1,25,7,22903,One Flew Over the Cuckoo's Nest (1975),[Drama]
4,17,1193,5,978158471,1,50,1,95350,One Flew Over the Cuckoo's Nest (1975),[Drama]
...,...,...,...,...,...,...,...,...,...,...
817419,5433,286,3,960240881,0,35,17,45014,Nemesis 2: Nebula (1995),"[Action, Sci-Fi, Thriller]"
817420,5556,2198,3,959445515,1,45,6,92103,Modulations (1998),[Documentary]
817421,5949,2198,5,958846401,1,18,17,47901,Modulations (1998),[Documentary]
817422,5675,2703,3,976029116,1,35,14,30030,Broken Vessels (1998),[Drama]


In [ ]:
ratingusers_data_test = pandas.merge(ratingusers_data, movies_test, on='movieid')
ratingusers_data_test

,userid,movieid,rating,timestamp,gender,age,occupation,zip,title,genre
0,1,2797,4,978302039,0,1,10,48067,Big (1988),"[Comedy, Fantasy]"
1,10,2797,5,979168267,0,35,1,95370,Big (1988),"[Comedy, Fantasy]"
2,18,2797,4,978152454,0,18,3,95825,Big (1988),"[Comedy, Fantasy]"
3,22,2797,3,978153058,1,18,15,53706,Big (1988),"[Comedy, Fantasy]"
4,23,2797,4,978461410,1,35,0,90049,Big (1988),"[Comedy, Fantasy]"
...,...,...,...,...,...,...,...,...,...,...
182780,4816,2309,4,965869069,0,45,6,04240,"Inheritors, The (Die Siebtelbauern) (1998)",[Drama]
182781,5334,127,1,960795494,0,56,13,46140,"Silence of the Palace, The (Saimt el Qusur) (1...",[Drama]
182782,5494,3530,4,959816296,0,35,17,94306,Smoking/No Smoking (1993),[Comedy]
182783,5851,3607,5,957756608,0,18,20,55410,One Little Indian (1973),"[Comedy, Drama, Western]"


## Part 1. Dataset + Dataloader

 Thêm movies_train, movies_test ba cột: 'id', giá trị chỉ mục đã đặt lại và 'img_path', đường dẫn tệp đầy đủ cho hình ảnh phim.

In [ ]:
# movies_train = pandas.read_csv('ml1m/dataset/movies_train.dat', engine='python',
#                          sep='::', names=['movieid', 'title', 'genre'], encoding='latin-1', index_col=False).set_index('movieid')

folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_train['id'] = movies_train.index
movies_train.reset_index(inplace=True)
movies_train['img_path'] = movies_train.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_train

,movieid,title,genre,id,img_path
0,1650,Washington Square (1997),[Drama],1650,ml1m/content/dataset/ml1m-images/1650.jpg
1,185,"Net, The (1995)","[Sci-Fi, Thriller]",185,ml1m/content/dataset/ml1m-images/185.jpg
2,1377,Batman Returns (1992),"[Action, Adventure, Comedy, Crime]",1377,ml1m/content/dataset/ml1m-images/1377.jpg
3,3204,"Boys from Brazil, The (1978)",[Thriller],3204,ml1m/content/dataset/ml1m-images/3204.jpg
4,1901,Dear Jesse (1997),[Documentary],1901,ml1m/content/dataset/ml1m-images/1901.jpg
...,...,...,...,...,...
3101,2539,Analyze This (1999),[Comedy],2539,ml1m/content/dataset/ml1m-images/2539.jpg
3102,3038,"Face in the Crowd, A (1957)",[Drama],3038,ml1m/content/dataset/ml1m-images/3038.jpg
3103,1832,Heaven's Burning (1997),"[Action, Drama]",1832,ml1m/content/dataset/ml1m-images/1832.jpg
3104,657,Yankee Zulu (1994),"[Comedy, Drama]",657,ml1m/content/dataset/ml1m-images/657.jpg


In [ ]:
folder_img_path = 'ml1m/content/dataset/ml1m-images'
movies_test['id'] = movies_test.index
movies_test.reset_index(inplace=True)
movies_test['img_path'] = movies_test.apply(lambda row: os.path.join(folder_img_path, f'{row.id}.jpg'), axis = 1)
movies_test

,movieid,title,genre,id,img_path
0,3397,"Great Muppet Caper, The (1981)","[Children's, Comedy]",3397,ml1m/content/dataset/ml1m-images/3397.jpg
1,2067,Doctor Zhivago (1965),"[Drama, Romance, War]",2067,ml1m/content/dataset/ml1m-images/2067.jpg
2,2651,Frankenstein Meets the Wolf Man (1943),[Horror],2651,ml1m/content/dataset/ml1m-images/2651.jpg
3,2989,For Your Eyes Only (1981),[Action],2989,ml1m/content/dataset/ml1m-images/2989.jpg
4,3415,"Mirror, The (Zerkalo) (1975)",[Drama],3415,ml1m/content/dataset/ml1m-images/3415.jpg
...,...,...,...,...,...
772,2309,"Inheritors, The (Die Siebtelbauern) (1998)",[Drama],2309,ml1m/content/dataset/ml1m-images/2309.jpg
773,2421,"Karate Kid, Part II, The (1986)","[Action, Adventure, Drama]",2421,ml1m/content/dataset/ml1m-images/2421.jpg
774,3255,"League of Their Own, A (1992)","[Comedy, Drama]",3255,ml1m/content/dataset/ml1m-images/3255.jpg
775,974,Algiers (1938),"[Drama, Romance]",974,ml1m/content/dataset/ml1m-images/974.jpg


##model

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# df_train và df_test là DataFrame chứa dữ liệu train và test của bạn
df_train = ratingusers_data_train
df_test = ratingusers_data_test

# Chọn các đặc trưng cần sử dụng
selected_features = ['gender', 'rating', 'age', 'occupation']
X_train = df_train[selected_features]
y_train = df_train['genre']
X_test = df_test[selected_features]
y_test = df_test['genre']

# Kiểm tra và xử lý giá trị NaN trong cột 'genre'
nan_indices_train = y_train.index[y_train.isna()]
nan_indices_test = y_test.index[y_test.isna()]

# Xóa các dòng có giá trị NaN trong cột 'genre'
y_train = y_train.drop(index=nan_indices_train)
X_train = X_train.drop(index=nan_indices_train)
y_test = y_test.drop(index=nan_indices_test)
X_test = X_test.drop(index=nan_indices_test)

# Chuyển đổi thể loại thành dạng số
genre_mapping = {genre: i for i, genre in enumerate(set(y_train.explode()))}
y_train_encoded = y_train.apply(lambda x: genre_mapping[x[0]])  # Lấy giá trị đầu tiên trong danh sách
y_test_encoded = y_test.apply(lambda x: genre_mapping[x[0]])  # Lấy giá trị đầu tiên trong danh sách

# Chuyển đổi thành tensor PyTorch
X_train = torch.FloatTensor(X_train[selected_features].astype(float).values)
y_train_encoded = torch.LongTensor(y_train_encoded.values)
X_test = torch.FloatTensor(X_test[selected_features].astype(float).values)
y_test_encoded = torch.LongTensor(y_test_encoded.values)

# Khởi tạo mô hình Neural Network đa lớp
class MultiLabelClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultiLabelClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_size),
            nn.Sigmoid()  # Sử dụng hàm kích hoạt Sigmoid để có xác suất đầu ra
        )

    def forward(self, x):
        return self.fc(x)

# Khởi tạo mô hình và bộ tối ưu hóa
input_size = X_train.shape[1]
output_size = len(set(y_train_encoded.numpy()))
model = MultiLabelClassifier(input_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Chuyển đổi y_train_encoded thành mã hóa one-hot
y_train_onehot = F.one_hot(y_train_encoded, num_classes=output_size)

# Chuyển đổi y_train_onehot thành kiểu dữ liệu float
y_train_onehot = y_train_onehot.float()

# Huấn luyện mô hình
epochs = 10
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train)

    # Chuyển đổi outputs thành kích thước phù hợp với y_train_onehot
    outputs = outputs.view(-1, output_size)

    loss = criterion(outputs, y_train_onehot)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Đánh giá mô hình trên tập test
    with torch.no_grad():
        model.eval()
        predictions = (model(X_test) > 0.3).float()  # Dự đoán với ngưỡng 0.3

        # Chuyển đổi predictions thành mã nhãn
        predicted_labels = predictions.argmax(dim=1)

        accuracy = accuracy_score(y_test_encoded.numpy(), predicted_labels.numpy())

    # In giá trị loss và accuracy mỗi epoch
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Accuracy: {accuracy}')

classification_rep = classification_report(y_test_encoded.numpy(), predicted_labels.numpy())

print(f'Accuracy: {accuracy*100}%')
print('Classification Report:')
print(classification_rep)


Epoch 1/10, Loss: 1.2638473510742188, Accuracy: 0.00048143994310255217
Epoch 2/10, Loss: 0.7796167731285095, Accuracy: 0.0541018136061493
Epoch 3/10, Loss: 0.4585030972957611, Accuracy: 0.06494515414284542
Epoch 4/10, Loss: 0.29887881875038147, Accuracy: 0.08898432584730695
Epoch 5/10, Loss: 0.23712770640850067, Accuracy: 0.012812867576661105
Epoch 6/10, Loss: 0.21834604442119598, Accuracy: 0.016429138058374593
Epoch 7/10, Loss: 0.2146245241165161, Accuracy: 0.026982520447520312
Epoch 8/10, Loss: 0.21469596028327942, Accuracy: 0.04245424952813415
Epoch 9/10, Loss: 0.2136346697807312, Accuracy: 0.06482479415706978
Epoch 10/10, Loss: 0.2096072882413864, Accuracy: 0.08540088081625954
Accuracy: 8.540088081625955%
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.05      0.00      0.00     11180
           2       0.00      0.00      0.00       325
           3       0.00      0.00      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.